# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

**Volume** : 112 pages, 39 exemples, 35 papiers

- 3.1. [**Validation croisée : évaluer les performances des estimateurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-evaluating-estimator-performance)<br/>([_Cross-validation: evaluating estimator performance_](https://scikit-learn.org/stable/model_selection.html#cross-validation-evaluating-estimator-performance))
    - **Volume** : 25 pages, 8 exemples, 8 papiers
    - 3.1.1. [**Calcul de métriques à validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#computing-cross-validated-metrics)<br/>([_computing-cross-validated-metrics_](https://scikit-learn.org/stable/model_selection.html#computing-cross-validated-metrics))
    - 3.1.2. [**Itérateurs de validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-iterators)<br/>([_Cross validation iterators_](https://scikit-learn.org/stable/model_selection.html#cross-validation-iterators))
    - 3.1.3. [**Une note sur le brassage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#a-note-on-shuffling)<br/>([_A note on shuffling_](https://scikit-learn.org/stable/model_selection.html#a-note-on-shuffling))
    - 3.1.4. [**Validation croisée et sélection de modèles**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-and-model-selection)<br/>([_cross-validation-and-model-selection_](https://scikit-learn.org/stable/model_selection.html#cross-validation-and-model-selection))
    - 3.1.5. [**Résultat du test de permutation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#permutation-test-score)<br/>([_Permutation test score_](https://scikit-learn.org/stable/model_selection.html#permutation-test-score))
- 3.2. [**Réglage des hyper-paramètres d'un estimateur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tuning-the-hyper-parameters-of-an-estimator)<br/>([_Tuning the hyper-parameters of an estimator_](https://scikit-learn.org/stable/model_selection.html#tuning-the-hyper-parameters-of-an-estimator))
    - **Volume** : 18 pages, 9 exemples, 3 papiers
    - 3.2.1. [**Recherche exhaustive en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#exhaustive-grid-search)<br/>([_Exhaustive Grid Search_](https://scikit-learn.org/stable/model_selection.html#exhaustive-grid-search))
    - 3.2.2. [**Optimisation aléatoire des paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#randomized-parameter-optimization)<br/>([_Randomized Parameter Optimization_](https://scikit-learn.org/stable/model_selection.html#randomized-parameter-optimization))
    - 3.2.3. [**Recherche de paramètres optimaux par halving successifs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#searching-for-optimal-parameters-with-successive-halving)<br/>([_Searching for optimal parameters with successive halving_](https://scikit-learn.org/stable/model_selection.html#searching-for-optimal-parameters-with-successive-halving))
    - 3.2.4. [**Conseils pour la recherche de paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tips-for-parameter-search)<br/>([_Tips for parameter search_](https://scikit-learn.org/stable/model_selection.html#tips-for-parameter-search))
    - 3.2.5. [**Alternatives à la recherche de paramètres par force brute**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#alternatives-to-brute-force-parameter-search)<br/>([_Alternatives to brute force parameter search_](https://scikit-learn.org/stable/model_selection.html#alternatives-to-brute-force-parameter-search))
- 3.3. [**Métriques et scoring : quantifier la qualité des prédictions**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#metrics-and-scoring-quantifying-the-quality-of-predictions)<br/>([_Metrics and scoring: quantifying the quality of predictions_](https://scikit-learn.org/stable/model_selection.html#metrics-and-scoring-quantifying-the-quality-of-predictions))
    - **Volume** : 63 pages, 19 exemples, 24 papiers
    - 3.3.1. [**Le paramètre `scoring` : définir les règles d'évaluation du modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#the-scoring-parameter-defining-model-evaluation-rules)<br/>([_The `scoring` parameter: defining model evaluation rules_](https://scikit-learn.org/stable/model_selection.html#the-scoring-parameter-defining-model-evaluation-rules))
    - 3.3.2. [**Métriques de classification**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#classification-metrics)<br/>([_Classification metrics_](https://scikit-learn.org/stable/model_selection.html#classification-metrics))
    - 3.3.3. [**Métriques de classement multi-étiquettes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#multilabel-ranking-metrics)<br/>([_Multilabel ranking metrics_](https://scikit-learn.org/stable/model_selection.html#multilabel-ranking-metrics))
    - 3.3.4. [**Métriques de régression**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#regression-metrics)<br/>([_Regression metrics_](https://scikit-learn.org/stable/model_selection.html#regression-metrics))
    - 3.3.5. [**Métriques de clustering**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#clustering-metrics)<br/>([_Clustering metrics_](https://scikit-learn.org/stable/model_selection.html#clustering-metrics))
    - 3.3.6. [**Estimateurs factices**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#dummy-estimators)<br/>([_Dummy estimators_](https://scikit-learn.org/stable/model_selection.html#dummy-estimators))
- 3.4. [**Courbes de validation : tracer des scores pour évaluer des modèles**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#validation-curves-plotting-scores-to-evaluate-models)<br/>([_Validation curves: plotting scores to evaluate models_](https://scikit-learn.org/stable/model_selection.html#validation-curves-plotting-scores-to-evaluate-models))
    - **Volume** : 6 pages, 3 exemples, 0 papiers
    - 3.4.1. [**Courbe de validation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#validation-curve)<br/>([_Validation curve_](https://scikit-learn.org/stable/model_selection.html#validation-curve))
    - 3.4.2. [**Courbe d'apprentissage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#learning-curve)<br/>([_Learning curve_](https://scikit-learn.org/stable/model_selection.html#learning-curve))